In [1]:
#load the packages
#import numpy
import pandas
#import io
import os
import glob
from sklearn.preprocessing import LabelEncoder
import numpy as np
#following required for LSTM
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from sklearn import metrics
import matplotlib.pyplot as plt
import keras_tuner as kt
import tensorflow as tf
#following requried for DTW_kNN
from sklearn.metrics import accuracy_score, log_loss
from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, KNeighborsTimeSeries
#set the working directory
os.chdir("/group/yleung/data/genotype_prediction/seperate_on_off/")

In [2]:
def df2array(df, YellowPage, fold):
    test_uid = YellowPage.loc[YellowPage.iloc[:,fold+1] == 'Test']['uid']
    train_uid = YellowPage.loc[YellowPage.iloc[:,fold+1] == 'Train']['uid']
    #subset the dataset
    test_sub = df.loc[df['uid'].isin(test_uid)]
    train_sub = df.loc[df['uid'].isin(train_uid)]
    #transform from long format to 3-d array, [sample, time_step, variable]
    test_list = [x for _, x in test_sub.groupby(['uid'])]
    test_values = np.array([d_f.values for d_f in test_list])
    train_list = [x for _, x in train_sub.groupby(['uid'])]
    train_values = np.array([d_f.values for d_f in train_list])
    #make sure all data is float
    train = np.asarray(train_values[:,:,:(train_values.shape[2]-1)]).astype(np.float32)
    test = np.asarray(test_values[:,:,:(test_values.shape[2]-1)]).astype(np.float32)
    return train, test

In [3]:
def LSTM_model(x, y):
    #design the model
    model = Sequential()
    model.add(LSTM(50, input_shape= x[0].shape[1:])) #training X shape
    #model.add(Dropout(0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ['accuracy'])
    #fit the model
    history = model.fit(x[0], y[0], epochs = 100, batch_size = 64, verbose = 0) #training X and training Y
    
    #predict the test
    test_pred = model.predict(x[1])
    #confusion matrix
    test_matrix = metrics.confusion_matrix(y[1], np.rint(test_pred))
    
    #break down
    tn = test_matrix[0, 0]
    tp = test_matrix[1, 1]
    fn = test_matrix[1, 0]
    fp = test_matrix[0, 1]
    #output metrics
    acc = metrics.accuracy_score(y[1], np.rint(test_pred)) #accuracy
    sns = metrics.recall_score(y[1], np.rint(test_pred)) #sensitivity = recall
    sps = tn / (tn + fp) #specificity
    prc = metrics.precision_score(y[1], np.rint(test_pred)) #precision
    kappa = metrics.cohen_kappa_score(y[1], np.rint(test_pred)) #Cohen's kappa
    auroc = metrics.roc_auc_score(y[1], np.rint(test_pred)) #area under ROC curve

    perform_metrics = [acc, sns, sps, prc, kappa, auroc]
    
    return  perform_metrics

In [4]:
def DTW_knn(x, y): 
    #design the model
    knn_clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, metric="dtw")
    knn_clf.fit(x[0], y[0])
    #predict the outcome
    predicted_labels = knn_clf.predict(x[1])
    #confusion matrix
    test_matrix = metrics.confusion_matrix(y[1], predicted_labels)
    
    #break down
    tn = test_matrix[0, 0]
    tp = test_matrix[1, 1]
    fn = test_matrix[1, 0]
    fp = test_matrix[0, 1]
    #output metrics
    acc = metrics.accuracy_score(y[1], predicted_labels) #accuracy
    sns = metrics.recall_score(y[1], predicted_labels) #sensitivity = recall
    sps = tn / (tn + fp) #specificity
    prc = metrics.precision_score(y[1], predicted_labels) #precision
    kappa = metrics.cohen_kappa_score(y[1], predicted_labels) #Cohen's kappa
    auroc = metrics.roc_auc_score(y[1], predicted_labels) #area under ROC curve

    perform_metrics = [acc, sns, sps, prc, kappa, auroc]
    
    return  perform_metrics


In [5]:
#make a function to load, preprocess and build the LSTM model
def CV(Data, Yellow_Page):
    #create a list to store the accuracy for each fold
    LSTM_fold_perm = []
    DTWkNN_fold_perm = []
    for k in range(1,11):
        #split the train 
        train, test = df2array(Data, Yellow_Page, k)
        #split the indenpendent and response variables
        X = (train[:, :, 2:], test[:, :, 2:])# X of train and test
        Y = (train[:, :, 0][:,0].astype(np.intc), test[:, :, 0][:,0].astype(np.intc))#Y of train and test
        #apply each model for the independent and response variables
        LSTM_fold_perm.append(LSTM_model(X, Y))
        DTWkNN_fold_perm.append(DTW_knn(X, Y))
        #print an indicator
        print("Fold Complete:", k)
    #calculate the mean and sd across folds
    cv_mean_sd = np.array([np.mean(LSTM_fold_perm, axis = 0),
                          np.std(LSTM_fold_perm, axis = 0),
                          np.mean(DTWkNN_fold_perm, axis = 0), 
                          np.std(DTWkNN_fold_perm, axis = 0)]) 
     
    ##convert the numpy array to pandas data frame
    #row names and column names
    row_names = ["LSTM" + x for x in [".mean", ".sd"]] + ["DTW" + x for x in [".mean", ".sd"]]
    col_names = ["Accuracy", "Sensitivity", "Specificity", "Precision", "Kappa", "AUROC"]
    #convert the numpy array to dataframe
    output = pandas.DataFrame(cv_mean_sd, columns=col_names, index = row_names)
    return(output)

In [6]:
#main function
#FUNCTION: preprocess the data
def main(fname, sec):
    #get the filename without dir and file extension
    prefix = 'py_input_data/'
    suffix = '.csv'
    f_core = fname[len(prefix):][:-len(suffix)] #parse the core identifier for each dataset
    #read the csv file to a data frame
    data_frame = pandas.read_csv(fname, sep = ",", decimal='.')
    #remove the an and location column
    dataset = data_frame.drop(['an', 'location', 'end', 'batch'], axis=1)
    #specify the cutting time point based on different time point
    #parse the time segment
    dataset_sub = dataset[dataset['start'] < sec]
    #encode the genotype label
    encoder = LabelEncoder()
    dataset_sub['genotype'] = encoder.fit_transform(dataset_sub['genotype']) # 0 = mut, 1 = WT
    
    #get the corresponding yellow page filename
    yellow_page_prefix = 'int_output/'
    yellow_page_suffix = '_kFolds_yellow_page.csv'
    yellow_page_file = yellow_page_prefix + f_core + yellow_page_suffix
    #read the yellow page
    yellow_page = pandas.read_csv(yellow_page_file, sep = ",", decimal='.')
   
    #apply the cross validation function
    Acc_Df = CV(dataset_sub, yellow_page)
    #set the output directory and filenames
    out_path = "output/" + f_core + "/"
    f_pre = f_core + "_" + str(sec)
    if not os.path.exists(out_path):
        os.makedirs(out_path)#create the output dir for each dataset
    #save the accuracy dataframe to an output csv file
    Acc_Df.to_csv(out_path + f_pre + '_py_10CV.csv', sep=',')

In [7]:
#main
input_files = glob.glob("py_input_data/*.csv")
time_sec = [5, 30, 60, 120, 300]
for f in input_files:
    for t in time_sec:
        main(f, t)

/tmp/ipykernel_42171/876398946.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_sub['genotype'] = encoder.fit_transform(dataset_sub['genotype']) # 0 = mut, 1 = WT
2023-05-23 17:51:53.015674: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 17:51:53.026751: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 24. Tune using inter_op_parallelism_threads for best performance.


Fold Complete: 1



KeyboardInterrupt

